In [ ]:
installPreReqs=False


In [ ]:
if installPreReqs:
    !pip install xgboost
    !pip install shap
    !pip install pandas
    !pip install numpy
    !pip install scipy
    !pip install matplotlib
    !pip install scipy
    !pip install seaborn
    !pip install catboost
    #!pip install skopt
    !pip install mlens
    !pip install xgboost


## Imports - Needs cleanup

In [ ]:
import xgboost
import pandas as pd
import numpy as np
import joblib
import scipy as sp
import time
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
import warnings
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV,cross_val_score,KFold
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,log_loss,f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,VotingClassifier,StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
#import mlens
#import catboost
from catboost import CatBoostClassifier
import random
import os
import itertools
from pathlib import Path
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0:1"
pd.options.display.max_colwidth = 60
#remove warnings for final output
warnings.filterwarnings('ignore')
seed=4242
random.seed(seed)
np.random.seed(seed)

## Runtime Options

In [ ]:
#enables model selection search
modelSelectionSearch=True
lowPriority=True
modelType=CatBoostClassifier(silent=True,task_type="CPU",eval_metric='F1', random_strength=.2, border_count=32,iterations=250,
                             bagging_temperature=.03,max_ctr_complexity=1,l2_leaf_reg=100,depth=6,learning_rate=.03)
#Degree of interaction features to create
degreeInteractions=0
#Try scaling features
scaleX=False
#Search stacking models
tryStacking=True

In [ ]:
def lowpriority():
    """ Set the priority of the process to below-normal."""

    import sys
    try:
        sys.getwindowsversion()
    except AttributeError:
        isWindows = False
    else:
        isWindows = True

    if isWindows:
        # Based on:
        #   "Recipe 496767: Set Process Priority In Windows" on ActiveState
        #   http://code.activestate.com/recipes/496767/
        import win32api,win32process,win32con

        pid = win32api.GetCurrentProcessId()
        handle = win32api.OpenProcess(win32con.PROCESS_ALL_ACCESS, True, pid)
        win32process.SetPriorityClass(handle, win32process.BELOW_NORMAL_PRIORITY_CLASS)
    else:
        import os

        os.nice(1)

In [ ]:
if(lowPriority):
    lowpriority()
    


## Read input data

In [ ]:
df = pd.read_csv(df) ## Use the CRT data


## First visit only

In [ ]:
df.head()

In [ ]:
firstVisit =df['time']==0
thirdMonth =df['time']==3
sixthMonth =df['time']==6

In [ ]:
df=df[firstVisit]
df.set_index('Patno',inplace=True)

## Data Cleanup

In [ ]:
#  Did not expect the data types in fields EDV/ESV, Gender, LVEF, LVEF_DC, NYHA, NYHA_Description, PRIMARY, PRIMARYA, Patient_Status, Position_LAO_View, Position_RAO_View, Randomization_Group, , centersize, cube root, group,

del df['LVEF']
del df['LVESV6'] #result set
del df['VT'] #use vt types
del df['bb'] #duplicate
del df['isch'] #duplicate
del df['Sample']
del df['EDV/ESV']
del df['normal_indicator'] #static value
del df['pstatus']
del df['Patient_Status'] #static
del df['Visit'] #same as time
del df['statuschange'] #empty
del df['Randomization_Group']
del df['resp'] #duplicate to response
del df['NYHA_Description']
del df['cube root']
del df['substudy']
del df['death']
del df['PRIMARY'] #manually divided
del df['PRIMARYA']
#del df['group']
del df['Position_RAO_View']
del df['Position_LAO_View']
del df['time'] #not needed
del df['lbbb'] #duplicate of history_of_left
del df['LVEF_DC'] #noise
del df['base_LVESV'] #remove from course 1
del df['qrs'] #i think this is an after measureement
del df['AV_INTERVAL_WITH_ATRIAL_PACING__'] #only available after crt
df['EDV_ESV'] = df.LVEDV/ df.LVESV #testing
df['LVEF'] = 100*(df.LVEDV - df.LVESV)/df.LVEDV
df['cube_root']=df.EDV_ESV**(1/3)

df['tachycardia']=df.apply(lambda x: 1 if x['hrrest']>100 else 0, axis=1)
df['bradycardia']=df.apply(lambda x: 1 if x['hrrest']<60 else 0, axis=1)

#assuming crp in ug not ng as paper suggests (due to ranges)
#setting normal cutoff at 10000ug based on
#https://www.mayoclinic.org/tests-procedures/c-reactive-protein-test/about/pac-20385228#:~:text=For%20a%20standard%20CRP%20test,testing%20to%20determine%20the%20cause.

df['high_crp']=df.apply(lambda x: 1 if x['crp']>10000 else 0, axis=1)


df['normalbp']=df.apply(lambda x: 1 if x['bpsys']<120 and x['bpdia']<80  else 0, axis=1)
df['s1hypertension']=df.apply(lambda x: 1 if (x['bpsys']>=130 and x['bpsys']<=139) or (x['bpdia']>=80 and x['bpdia']<=89)  else 0, axis=1)
df['s2hypertension']=df.apply(lambda x: 1 if x['bpsys']>=140 or x['bpdia']>=90  else 0, axis=1)
##using classes from 
#https://www.cdc.gov/obesity/adult/defining.html#:~:text=If%20your%20BMI%20is%2018.5,falls%20within%20the%20obese%20range.

df['obesity_c1']=df.apply(lambda x: 1 if x['bmi']>=30 and x['bmi']<35  else 0, axis=1)
df['obesity_c2']=df.apply(lambda x: 1 if x['bmi']>=35 and x['bpdia']<40  else 0, axis=1)
df['obesity_c3']=df.apply(lambda x: 1 if x['bmi']>=40 else 0, axis=1)
df['underweight']=df.apply(lambda x: 1 if x['bmi']<18.5 else 0, axis=1)
df['normalweight']=df.apply(lambda x: 1 if x['bmi']>=18.5 and x['bmi']<25 else 0, axis=1)
#df['crt_response']=df.apply(lambda x: 1 if abs(x['lvesv_delta'])>15 else 0, axis=1)
#normal qrs width = 120 > = wide
#should this be 145? 
#https://watermark.silverchair.com/eup258.pdf?token=AQECAHi208BE49Ooan9kkhW_Ercy7Dm3ZL_9Cf3qfKAc485ysgAAAq0wggKpBgkqhkiG9w0BBwagggKaMIIClgIBADCCAo8GCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMRWMvb5zxqoL3gsiSAgEQgIICYEBKxKn7lMiuicw1bf-sK96doZtlZfO_UITAQA9WmknUC3ab8yEoY_RSOfYP0BeWNCjXWabG9U_CQk05x_-HbOXozkMWvXuYDi2VvrF5XQRAyrSa3EAGp9DRtmgVt2btKil-ACIPBCtNL-saaq-6EGdo5EPB7f-NdTlqWvwbdngOdTUax-hTagz4jB1OYE0CPjMAcmafbEFjvNagVYrQB4j6BEo434Skq33YXNXgaqfCW5a1NdqQC07MXU1P3ASq_xL-BbJSGgv_67dh0SxIVPqftjShBTz6OYbcUtp1PrVClsehK4jTlGHYrqoJn-fnZOC_8ZnEaTEUlKgepco_sYvhiiGGkPDTwI2YwRC0o3j7MQzBn3di2bVWs1xThW2PyA9L6Eb4cpOe2LZHXnCp12P_3nZSzmsSMoOHhXfuy7CV_6m7IEY0cumjua2bqlpiIwpGNXtUWllBIaol5897lFJIq6NFJsqeBSHg0FOoSDvd7HhBHb7JHkhuZ_uQxcL-YJpZUBQ17bwlZcXtqsWlBZp00IR3o-mcteIR8gMDMGwuQErbXcyd3erE8AfM-B-UOir0NJQqtAuO_SvXiuePg6JKVxbOtSyobjQ1rqPQ5cbsxtm-sFXDHJEallvJYcdPsHxPueBsvTQ9pR1jOTfq-UFMRSNVSwmNT2yuFwXlTUkVmYFBoTi7X-WuJaBoH2jvo_U2AQADeYz4hk-bHtthaXIqAo6k1Vc61uhhJyr_7Xt5pdZcpNDZJblO8VFzsvW4l9-7FM2xs3MWAzF2NYatB7bKdCZURbXF-PBa3ZvdKZqr
#df['normalQRS']=df.apply(lambda x: 1 if x['QRS_WIDTH_WITHOUT_ATRIAL_PACING']>70 and x['QRS_WIDTH_WITHOUT_ATRIAL_PACING']<110 else 0, axis=1)
df['wideQRSwidth']=df.apply(lambda x: 1 if x['QRS_WIDTH_WITHOUT_ATRIAL_PACING']>120 else 0, axis=1)
df['optimalAVnoPacing']=df.apply(lambda x: 1 if x['AV_INTERVAL_WITHOUT_ATRIAL_PACIN']>=100 and  x['AV_INTERVAL_WITHOUT_ATRIAL_PACIN']<=150 else 0, axis=1)
#feature removal testing
del df['hrrest'] #use tachy/brady feature
#del df['height2'] #reduce multicoll.
#del df['weight2'] #reduce multicoll.
del df['MR_dP_dt__mm_Hg_s_1_'] #missing data too large
del df['MR_Area__cm2_'] #missing data too large
del df['MR_Width__mm_'] #missing data too large

#del df['QRS_WIDTH_WITHOUT_ATRIAL_PACING'] #use wide measurement
#del df['AV_INTERVAL_WITHOUT_ATRIAL_PACIN'] #use optimal feature

##experiment
#del df['bmi'] #multicol

## One hot encode columns

In [ ]:
def upperColumn(df, col):
    df[col] = df[col].str.upper()
    return df

def oneHot(df, col, prefix):
    df=upperColumn(df,col)
    Oneht = pd.get_dummies(df[col], prefix=prefix, drop_first=True)
    df = pd.concat([df, Oneht], axis=1)
    del df[col]
    return df


In [ ]:
df=oneHot(df,'group','grp_')
#df=oneHot(df,'Position_RAO_View','PosRAO_')
#df=oneHot(df,'Position_LAO_View','PosLAO_')
df=oneHot(df,'NYHA','NYHA_')
#df=oneHot(df,'LVEF_DC','LVEF_DC_')
df=oneHot(df,'Gender','Gender_')
df=oneHot(df,'centersize','centersize_')
df=oneHot(df,'AT Type','AT_')
df=oneHot(df,'VT Type','VT_')
#cleanup columns


## get CRT score based on article methodology

In [ ]:
def getCRTScore(row):
    crtScore=0
    if row['tnfrii']>=7090 :
        crtScore+=1
    if row['st2']>=23721 :
        crtScore+=1
    if row['crp']>=7381:
        crtScore+=1
    if row['mmp2']>=982000 :
        crtScore+=1
    return crtScore


In [ ]:
df['crt_score']=df.apply(lambda row: getCRTScore(row),axis=1)

In [ ]:
#.68 pruning at .25 pearsonR
#drop column
###AV INTERVAL dropped after imputation
#df.drop('AV_INTERVAL_WITHOUT_ATRIAL_PACIN', axis=1, inplace=True)
#df.drop('EDV/ESV', axis=1, inplace=True)
#df.drop('cube root', axis=1, inplace=True)
#df.drop('bmi', axis=1, inplace=True)
#absLVESV/LVEDV
#df.drop('LVEDV', axis=1, inplace=True) #hurt model


## Function to Impute using XGB (Unused)

## Function to Perform Stratified KFold (5) on model

In [ ]:
#Use stratified kfold cross validation of 5 splits

def cvModel(model,label):
    print('5-fold cross validation:\n')
    #kf=KFold(n_splits=5, random_state=1)
    kf = StratifiedKFold(n_splits=5, random_state=1)
    scores = cross_val_score(model, X_train,y_train, cv=kf, scoring ='f1')
    recall_score=cross_val_score(model, X_train,y_train, cv=kf, scoring ='recall')
    print(" f1: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    return {'avg':scores.mean(), 'std': scores.std()}

## show missing values

In [ ]:
pd.options.display.max_rows = 4000

print(df.isnull().sum())

## Impute or drop missing values

In [ ]:
#This did not improve the model, but more time should be used to tune the approach.
#Attempt to predict missing values w/ regression & categorical w/ most frequent

#df=predictMissingCon(df,'timp4')
#df=predictMissingCon(df,'sgp130')
#df=predictMissingCon(df,'sil2ra')
#df=predictMissingCon(df,'hrrest')
#df=predictMissingCon(df,'qol_score')
#df=predictMissingCon(df,'AV_INTERVAL_WITHOUT_ATRIAL_PACIN')
#df=predictMissingCon(df,'QRS_WIDTH_WITHOUT_ATRIAL_PACING')
#df=predictMissingCon(df,'PR_INTERVAL_WITHOUT_ATRIAL_PACIN')
#df=predictMissingCon(df,'__Minute_Walk')
#df=predictMissingCon(df,'QRS_WIDTH_WITHOUT_ATRIAL_PACING') #makes model worse
#categorical
#impute categorical values with most frequent
print("Missing Records before imputation")
print(df.isnull().sum())
imputer2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
n = ["cabg","pci"]
for n in n:
    imputer2 = imputer2.fit(df[[n]])
    df[n] = imputer2.transform(df[[n]])
print("Dropping remaining missing values")
#drop AV aftger predicting PR
#df.drop('AV_INTERVAL_WITHOUT_ATRIAL_PACIN', axis=1, inplace=True)
print("Missing Records after imputation")
print(df.isnull().sum())
df=df.dropna()

## Set labels and features

In [ ]:
y = df['response']
#remove unwanted columns
X = df.loc[:, df.columns.difference(['response', 'Patno','2 clusters','3 clusters','4 clusters','lvesv_delta','change','LVESV6','death'])]
#X = df.loc[:, df.columns.difference(['crt_response', 'Patno','2 clusters','3 clusters','4 clusters','lvesv_delta','LVEF','cube root','Pulse','LVEDV','bmi'])]
feature_names=X.columns
print(feature_names)
y=pd.DataFrame(y)
X = pd.DataFrame(X, columns =feature_names)


## Create Interactions

In [ ]:
if (degreeInteractions>0):
    print("Adding "+str(degreeInteractions)+" degree interactions")
    X = PolynomialFeatures(degreeInteractions, interaction_only=True, include_bias=False).fit_transform(X)


## Scale Data

In [ ]:
#StandardScaler seems to perform best on data
if scaleX :
    #scaler = MinMaxScaler()
    scaler=RobustScaler()
    print("Scaling data")
    #scaler=StandardScaler()
    X=scaler.fit_transform(X)

## Split Train & Test Set

In [ ]:
# Splitting Dataset for training and testing
from sklearn.model_selection import train_test_split
# Split the dataset using a 80:20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=5, stratify=y, shuffle=True)

## Confusion Matrix Function

In [ ]:
#from Anamul EDA notebook

# Function to automate the fitting, evaluation and confusion matrix
def classifier(model, feature_train, label_train, feature_test, label_test):
    model.fit(feature_train,label_train)
    label_predict = model.predict(feature_test)
    print(classification_report(label_test,label_predict))
    #Confusion matrix in the same way
    cf_matrix = confusion_matrix(label_test, label_predict)
    sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')
    #print( cf_matrix)
    accuracy=metrics.accuracy_score(label_test, label_predict)
    precision= metrics.precision_score(label_test, label_predict)
    recall=metrics.recall_score(label_test, label_predict)
    roc_auc=metrics.roc_auc_score(label_test, label_predict)
    f1=metrics.f1_score(label_test, label_predict, average='weighted')
    #print("Accuracy:",metrics.accuracy_score(label_test, label_predict))
    #print("Precision:",metrics.precision_score(label_test, label_predict))
    #print("Recall:",metrics.recall_score(label_test, label_predict))
    #pl.show()
    return (accuracy,precision,recall,f1,roc_auc)

## Search classifiers for best AUC

In [ ]:
#select best model based on weighted F1 Score of train cv

def modelSearch(classifiers):
   
    estimatorResults=[]
    best_model_f1=0
    cvF1Avg=0
    cvF1Std=0
    skf=StratifiedKFold(shuffle=True, n_splits=5)
    display(Markdown("\n **Searching for best model...**\n\n"))
    for clf in classifiers:
            #find features
            rfecv = RFECV(estimator=clf, step=1, cv=StratifiedKFold(5),
            scoring='roc_auc')
            rfecv.fit(X_train, y_train.values.ravel())
            #end find features
            name = clf.__class__.__name__
            display(Markdown("\n **"+name+"**"))
            display(Markdown("\n **Train Data Results**\n\n"))
            #cvScore=cvModel(clf,name)
            scores=cross_val_score(clf, X_train, y_train, scoring='f1', cv=skf)
            clf.fit(X_train, y_train.values.ravel())
            pred=clf.predict(X_test)
            model_f1=f1_score(y_test, pred, average='weighted')
            if(model_f1>best_model_f1):
                best_model_f1=model_f1
                #best_clf=clf
            #if(cvScore['avg']>cvF1Avg):
            #    cvF1Avg=cvScore['avg']
            #    best_clf=clf
            #print(classification_report(y_test, pred))
            display(Markdown("\n **Test Data Results**\n\n"))
            testResult=classifier(clf,X_train,y_train,X_test,y_test)
            testAcc=testResult[0]
            testPrecision=testResult[1]
            testRecall=testResult[2]
            testF1=testResult[3]
            testAUC=testResult[4]
            estimatorResults.append((name,feature_names,clf,scaleX,degreeInteractions,scores,scores.mean(),scores.std(),testAcc,testPrecision,testRecall,testF1,testAUC))
            print('TrainMean:'+str(scores.mean()))
            print('TrainStd:'+str(scores.std()))
    return estimatorResults

## Grid Search for RandomForest Hyperparams

In [ ]:
#Grid Search for GradientBoostingClassifier
modelFileName=Path('models\RF.pk1')

if modelFileName.is_file():
    print("Loading model from disk...")
    rfBestModel=joblib.load(modelFileName)
else:
    #Grid Search for RF

    # Create the parameter grid based on the results of random search 
    param_grid = {
        'bootstrap': [True],
        'max_depth': [80, 90, 100, 110],
        'max_features': [2, 3,4,5,6,7,8,9,10,11,12,13,14,15],
        'min_samples_leaf': [3, 4, 5],
        'min_samples_split': [8, 10, 12],
        'n_estimators': [100, 200, 300, 1000],
        'random_state':[42]
    }
    skf=StratifiedKFold(shuffle=True, n_splits=5)
    rf = RandomForestClassifier()
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                              cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    grid_search.best_params_
    {'bootstrap': True,
     'max_depth': 80,
     'max_features': 3,
     'min_samples_leaf': 5,
     'min_samples_split': 12,
     'n_estimators': 100}
    rfBestModel = grid_search.best_estimator_
    joblib.dump(rfBestModel, modelFileName)
print(rfBestModel)


## SVC Hyperparameter Tuning

In [ ]:
from sklearn.pipeline import make_pipeline
#Grid Search for SVC
modelFileName=Path('models\SVC.pk1')

if modelFileName.is_file():
    print("Loading model from disk...")
    svcBestModel=joblib.load(modelFileName)
else:

    # Create the parameter grid based on the results of random search 
    param_grid = {
        'kernel': ['linear'],
        'gamma': ['auto','scale'],
        'probability': [True,False],
        'C':[1,52,10],
        'degree':[3,8],
        'random_state':[42]
    }
    skf=StratifiedKFold(shuffle=True, n_splits=5)
    X_train_tmp=X_train.copy()
    scaler=StandardScaler()
    print("Scaling data")
    #scaler=StandardScaler()
    X_train_tmp=scaler.fit_transform(X_train_tmp)
    
    grid_search = GridSearchCV(SVC(),
                                 param_grid,
                                 cv=skf,
                                 verbose=5,
                                 n_jobs = -1,
                                scoring='roc_auc')
    # Fit the grid search to the data
    grid_search.fit(X_train_tmp, y_train)
    svcBestModel = grid_search.best_estimator_
    joblib.dump(svcBestModel, modelFileName)
print(svcBestModel)


## knn hyperparameter tuning

In [ ]:
#Grid Search for KNN
modelFileName=Path('models\KNN.pk1')

if modelFileName.is_file():
    print("Loading model from disk...")
    knnBestModel=joblib.load(modelFileName)
else:
    #Grid Search for RF

    # Create the parameter grid based on the results of random search 
    param_grid = {
        'n_neighbors': [5,10,15,20,25],
        'weights': ['uniform','distance'],
        'algorithm': ['ball_tree','kd_tree','brute'],
        'leaf_size': [15,30 ,60,90],
        'p': [1,2],
    }
    skf=StratifiedKFold(shuffle=True, n_splits=5)
    knn = KNeighborsClassifier()
    grid_search = GridSearchCV(estimator = knn, param_grid = param_grid, 
                              cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    knnBestModel = grid_search.best_estimator_
    joblib.dump(knnBestModel, modelFileName)
print(knnBestModel)


## XGBoost Hyperparam tuning

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [ ]:
#Grid Search for GradientBoostingClassifier
modelFileName=Path('models\XGB.pk1')

if modelFileName.is_file():
    print("Loading model from disk...")
    xgBestModel=joblib.load(modelFileName)
else:
    #Grid Search for XGB

    # Create the parameter grid based on the results of random search 
    param_grid = {
        'n_estimators': [400, 700, 1000],
        'learning_rate':[.25,.50,.1,.15,.2,.25,.30],
        'colsample_bytree': [0.7, 0.8],
        'max_depth': [3,4,5,6,7,8,9,10],
        'reg_alpha': [1.1, 1.2, 1.3],
        'reg_lambda': [1.1, 1.2, 1.3],
        'subsample': [0.7, 0.8, 0.9],
        'eval_metric':['auc'],
        'use_label_encoder':[False],
        #'tree_method':['gpu_hist'],
        #'gpu_id':[-1],
        #'predictor':['gpu_predictor']
    }
    skf=StratifiedKFold(shuffle=True, n_splits=5)
    xg = XGBClassifier()
    grid_search = GridSearchCV(estimator = xg, param_grid = param_grid, 
                              cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc',)
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    xgBestModel = grid_search.best_estimator_
    joblib.dump(xgBestModel, modelFileName)
print(xgBestModel)


## Grid Search for GradientBoostingClassifer Hyperparams

In [ ]:
#Grid Search for GradientBoostingClassifier
modelFileName=Path('models\GBC.pk1')
#remove not
if modelFileName.is_file():
    print("Loading model from disk...")
    gbcBestModel=joblib.load(modelFileName)
else:
    # Create the parameter grid based on the results of random search 
    param_grid = {
        'max_depth': [80, 90, 100, 110],
        'max_features': [2, 3],
        'min_samples_leaf': [3, 4, 5],
        'min_samples_split': [8, 10, 12],
        'n_estimators': [100, 200, 300, 1000]
    }
    skf=StratifiedKFold(shuffle=True, n_splits=5)
    gb = GradientBoostingClassifier()
    grid_search = GridSearchCV(estimator = gb, param_grid = param_grid, 
                              cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    grid_search.best_params_
    {'max_depth': 80,
     'max_features': 3,
     'min_samples_leaf': 5,
     'min_samples_split': 12,
     'n_estimators': 100}
    gbcBestModel = grid_search.best_estimator_
    joblib.dump(gbcBestModel, modelFileName)
print(gbcBestModel)


## Grid Search for AdaBoost

In [ ]:
#Grid Search for Adaboost
#modelFileName=Path('models\ada.pk1')
modelFileName=Path('C:\\Users\\Doug\\explain\\models\\ada.pk1')
#remove not
if modelFileName.is_file():
    print("Loading model from disk...")
    adaBestModel=joblib.load(modelFileName)
else:
    # Create the parameter grid based on the results of random search 
    param_grid = {
        'n_estimators': [25,50,100, 200, 300, 1000],
        'learning_rate':[.25,.50,.1,.15,.2,.25,.30]
    }
    skf=StratifiedKFold(shuffle=True, n_splits=5)
    gb = AdaBoostClassifier()
    grid_search = GridSearchCV(estimator = gb, param_grid = param_grid, 
                              cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    adaBestModel = grid_search.best_estimator_
    joblib.dump(adaBestModel, modelFileName)
print(adaBestModel)


## Grid Search for CatBoost Hyperparams

In [ ]:
#Grid search for CB, parameters are for second tuning round.  Initial parameters were wider.
modelFileName=Path('models\CB.pk1')
#remove not
if modelFileName.is_file():
    print("Loading model from disk...")
    cbBestModel=joblib.load(modelFileName)
else:
    param_grid = {'depth':[3,1,2,6,4,5,7,8,9,10],
                  'iterations':[30,50,100,250,500],
                  'learning_rate':[0.03,0.01,0.001],
                  'l2_leaf_reg':[3,1,5,10,100],
                  'random_strength':[0.2,0.5,0.8],
                  'random_seed': [1],
                  'logging_level':['Silent'],
                  'od_type' : ["Iter"],
                  'od_wait' : [100]
                  }

    cb = CatBoostClassifier(task_type="CPU")
    skf=StratifiedKFold(shuffle=True, n_splits=3)
    grid_search = GridSearchCV(estimator = cb, param_grid = param_grid, 
                              cv = skf, n_jobs = -1, verbose = 2, scoring='roc_auc')
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    cbBestModel = grid_search.best_estimator_
    joblib.dump(cbBestModel, modelFileName)



## Test Classifers

## Stacking Classifier

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
#estimator=[('lda',LinearDiscriminantAnalysis()),('cb',cbBestModel),('gbc',gbcBestModel),('rf',rfBestModel),
#                   ('knn',KNeighborsClassifier(5))                   ,('ada',AdaBoostClassifier(LogisticRegression()))
#                   ,('xgb',xgBestModel),('gnb',GaussianNB()),('svc',SVC())]
#Train CV Mean Score: 0.7894 +/- 0.1264
#test .70

#train Train CV Mean Score: 0.7836 +/- 0.1011
#test .6920

#Train CV Mean Score: 0.7804 +/- 0.0122
#Test ROC_AUC:0.7046677215189874
xgbModel=xgBestModel
    #knn
knnModel= knnBestModel
    #svc
svcModel=svcBestModel
    #rf
rfModel=rfBestModel
    #gbc
gbcModel=gbcBestModel
    #lda
ldaModel= LinearDiscriminantAnalysis()
    #mlp
mlpModel= MLPClassifier(random_state=42, max_iter=2000)
    #logisticreg
lrModel= LogisticRegression(max_iter=120000)
##pipelines
mlpPipe = Pipeline([
        ('scale', StandardScaler()),
      #  ('reduce_dims', PCA(n_components=4)),
        ('mlp', mlpModel)])
rfPipe = Pipeline([
        ('scale', StandardScaler()),
       # ('reduce_dims', PCA(n_components=4)),
        ('rf', rfModel)])
svcPipe = Pipeline([
        ('scale', StandardScaler()),
        #('reduce_dims', PCA(n_components=4)),
        ('svc', svcModel)])
knnPipe = Pipeline([
        ('scale', StandardScaler()),
        #('reduce_dims', PCA(n_components=4)),
        ('knn', knnModel)])
ldaPipe = Pipeline([
        ('scale', StandardScaler()),
       # ('reduce_dims', PCA(n_components=4)),
        ('lda', ldaModel)])
lrPipe = Pipeline([
        ('scale', RobustScaler()),
      #  ('reduce_dims', PCA(n_components=4)),
        ('lda', ldaModel)])
cbPipe = Pipeline([
        #('scale', RobustScaler()),
      #  ('reduce_dims', PCA(n_components=4)),
         ('rfe',RFECV(cbBestModel)),
        ('cb', cbBestModel)])
adaPipe = Pipeline([
        #('scale', RobustScaler()),
      #  ('reduce_dims', PCA(n_components=4)),
     #   ('rfe',RFECV(cbBestModel)),
        ('ada', adaBestModel)])



adaBestModel
##pipelines
estimator=[('lda',ldaPipe),
           ('cb',cbBestModel),
           ('gbc',gbcModel),
           ('rf',rfPipe),
           ('xgb',xgbModel),
           #('bagging',BaggingClassifier(n_estimators=10)),
           ('svc',svcPipe),
   #        ('knn',knnPipe),
           ('mlp',mlpPipe),
        #   ('lr',lrPipe),
           ('ada',adaPipe)
           ]


clf = StackingClassifier(estimators=estimator,passthrough=True,final_estimator=lrPipe)


In [ ]:
skf=StratifiedKFold(shuffle=True, n_splits=5,random_state=seed)
scores=cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=skf)

In [ ]:
print("Train CV Mean Score: "+str(round(scores.mean(),4))+" +/- "+str(round(scores.std()*2,4)))
display(Markdown("\n **Test Set Results**\n\n"))
testResult=classifier(clf,X_train,y_train,X_test,y_test)


In [ ]:
#train .7751
clf.score(X_test,y_test)
stkCLF=clf

## Stacking classsifer ROC

In [ ]:
y_pred=clf.predict_proba(X_test)[:,1]
ypred=y_pred
X_valid=X_test.copy()
X_valid.loc[:,"Prediction"]=y_pred
#rounded prediction
X_valid.loc[:,"prediction"]=np.round(y_pred)
X_valid.loc[:,"response"]=y_test
crt_filter=[]
crt_df=[]
tn=[None,None,None,None,None]
fp=[None,None,None,None,None]
tp=[None,None,None,None,None]
fn=[None,None,None,None,None]
for x in range(5):
    crt_filter.append(X_valid['crt_score']==x)
    crt_df.append(X_valid[crt_filter[x]])
    tn[x], fp[x], fn[x], tp[x] = confusion_matrix(crt_df[x]['response'], crt_df[x]['prediction']).ravel()

In [ ]:
def show_roc_curve(crt_score='all'):
    if crt_score == 'all':
        fpr, tpr, _ = metrics.roc_curve(y_test,  ypred)
        auc = metrics.roc_auc_score(y_test,  ypred)
        auc=round(auc,4)
        plt.plot(fpr,tpr,label="ALL auc="+str(auc),color='red')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.plot([0, 1], [0, 1],linestyle='--')
        #new
        for crt_score in range(5):
            crt_score
            fpr, tpr, _ = metrics.roc_curve(crt_df[crt_score]['response'],  crt_df[crt_score]['prediction'])
            auc = metrics.roc_auc_score(crt_df[crt_score]['response'],  crt_df[crt_score]['prediction'])
            auc=round(auc,4)
            plt.plot(fpr,tpr,label="CRT "+str(crt_score)+" auc="+str(auc))
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.plot([0, 1], [0, 1],linestyle='--')
            plt.title("ROC_AUC by CRT Score")
        plt.legend(loc=4)
        plt.show()
        #plt.title("ROC_AUC Full Validation Set")
        #plt.legend(loc=4)
        #plt.show()
    else: 
        fpr, tpr, _ = metrics.roc_curve(crt_df[crt_score]['response'],  crt_df[crt_score]['prediction'])
        auc = metrics.roc_auc_score(crt_df[crt_score]['response'],  crt_df[crt_score]['prediction'])
        auc=round(auc,4)
        plt.plot(fpr,tpr,label="Stacked Model auc="+str(auc))
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.plot([0, 1], [0, 1],linestyle='--')
        plt.title("ROC_AUC for CRT Score:"+str(crt_score))
        plt.legend(loc=4)
        plt.show()
        
show_roc_curve('all')    
#show_roc_curve(0)
#show_roc_curve(1)
#show_roc_curve(2)
#show_roc_curve(3)
#show_roc_curve(4)

## Voting Classifer

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
#estimator=[('lda',LinearDiscriminantAnalysis()),('cb',cbBestModel),('gbc',gbcBestModel),('rf',rfBestModel),
#                   ('knn',KNeighborsClassifier(5))                   ,('ada',AdaBoostClassifier(LogisticRegression()))
#                   ,('xgb',xgBestModel),('gnb',GaussianNB()),('svc',SVC())]
#Train CV Mean Score: 0.7894 +/- 0.1264
#test .70

#train Train CV Mean Score: 0.7836 +/- 0.1011
#test .6920

#Train CV Mean Score: 0.7804 +/- 0.0122
#Test ROC_AUC:0.7046677215189874
xgbModel=xgBestModel
    #knn
knnModel= knnBestModel
    #svc
svcModel=svcBestModel
    #rf
rfModel=rfBestModel
    #gbc
gbcModel=gbcBestModel
    #lda
ldaModel= LinearDiscriminantAnalysis()
    #mlp
mlpModel= MLPClassifier(random_state=42, max_iter=2000)
    #logisticreg
lrModel= LogisticRegression(max_iter=120000)
##pipelines
mlpPipe = Pipeline([
        ('scale', StandardScaler()),
      #  ('reduce_dims', PCA(n_components=4)),
        ('mlp', mlpModel)])
rfPipe = Pipeline([
        ('scale', StandardScaler()),
       # ('reduce_dims', PCA(n_components=4)),
        ('rf', rfModel)])
svcPipe = Pipeline([
        ('scale', StandardScaler()),
        #('reduce_dims', PCA(n_components=4)),
        ('svc', svcModel)])
knnPipe = Pipeline([
        ('scale', StandardScaler()),
        #('reduce_dims', PCA(n_components=4)),
        ('knn', knnModel)])
ldaPipe = Pipeline([
        ('scale', StandardScaler()),
       # ('reduce_dims', PCA(n_components=4)),
        ('lda', ldaModel)])
lrPipe = Pipeline([
        ('scale', RobustScaler()),
      #  ('reduce_dims', PCA(n_components=4)),
        ('lda', ldaModel)])
cbPipe = Pipeline([
        #('scale', RobustScaler()),
      #  ('reduce_dims', PCA(n_components=4)),
         ('rfe',RFECV(cbBestModel)),
        ('cb', cbBestModel)])
adaPipe = Pipeline([
        #('scale', RobustScaler()),
      #  ('reduce_dims', PCA(n_components=4)),
     #   ('rfe',RFECV(cbBestModel)),
        ('ada', adaBestModel)])


adaBestModel
##pipelines
estimator=[('lda',ldaPipe),
           ('cb',cbBestModel),
           ('gbc',gbcModel),
           ('rf',rfPipe),
           ('xgb',xgbModel),
           #('bagging',BaggingClassifier(n_estimators=10)),
           ('svc',svcPipe),
        #   ('knn',knnPipe),
           ('mlp',mlpPipe),
           ('lr',lrPipe),
           ('ada',adaPipe)
           ]

#clf = StackingClassifier(estimators=estimator,final_estimator=lrPipe)

clf = VotingClassifier(estimators=estimator, voting='soft')


In [ ]:
skf=StratifiedKFold(shuffle=True, n_splits=5,random_state=seed)
scores=cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=skf)

In [ ]:
print("Train CV Mean Score: "+str(round(scores.mean(),4))+" +/- "+str(round(scores.std()*2,4)))
display(Markdown("\n **Test Set Results**\n\n"))
testResult=classifier(clf,X_train,y_train,X_test,y_test)


In [ ]:
y_pred=clf.predict_proba(X_test)[:,1]
ypred=y_pred

In [ ]:
auc=metrics.roc_auc_score(y_test,y_pred)


In [ ]:
ypred_base=[None]
x=0
for est in estimator:
    print("Testing Classifer:"+est[0])
    _=est[1].fit(X_train, y_train)
    scores=cross_val_score(_, X_train, y_train, scoring='roc_auc', cv=skf)
    ypred_base.append(est[1].predict_proba(X_test)[:,1])
    print(np.mean(scores))
    x+=1

## VotingClassifer ROC AUC Curve

In [ ]:
X_valid=X_test.copy()
X_valid.loc[:,"Prediction"]=y_pred
#rounded prediction
X_valid.loc[:,"prediction"]=np.round(y_pred)
X_valid.loc[:,"response"]=y_test
crt_filter=[]
crt_df=[]
tn=[None,None,None,None,None]
fp=[None,None,None,None,None]
tp=[None,None,None,None,None]
fn=[None,None,None,None,None]
for x in range(5):
    crt_filter.append(X_valid['crt_score']==x)
    crt_df.append(X_valid[crt_filter[x]])
    tn[x], fp[x], fn[x], tp[x] = confusion_matrix(crt_df[x]['response'], crt_df[x]['prediction']).ravel()

## ROC Curves by CRT Score

In [ ]:
def show_roc_curve(crt_score='all'):
    if crt_score == 'all':
        fpr, tpr, _ = metrics.roc_curve(y_test,  ypred)
        auc = metrics.roc_auc_score(y_test,  ypred)
        auc=round(auc,4)
        plt.plot(fpr,tpr,label="ALL auc="+str(auc),color='red')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.plot([0, 1], [0, 1],linestyle='--')
        #new
        for crt_score in range(5):
            crt_score
            fpr, tpr, _ = metrics.roc_curve(crt_df[crt_score]['response'],  crt_df[crt_score]['prediction'])
            auc = metrics.roc_auc_score(crt_df[crt_score]['response'],  crt_df[crt_score]['prediction'])
            auc=round(auc,4)
            plt.plot(fpr,tpr,label="CRT "+str(crt_score)+" auc="+str(auc))
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.plot([0, 1], [0, 1],linestyle='--')
            plt.title("ROC_AUC by CRT Score")
        plt.legend(loc=4)
        plt.show()
        #plt.title("ROC_AUC Full Validation Set")
        #plt.legend(loc=4)
        #plt.show()
    else: 
        fpr, tpr, _ = metrics.roc_curve(crt_df[crt_score]['response'],  crt_df[crt_score]['prediction'])
        auc = metrics.roc_auc_score(crt_df[crt_score]['response'],  crt_df[crt_score]['prediction'])
        auc=round(auc,4)
        plt.plot(fpr,tpr,label="Stacked Model auc="+str(auc))
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.plot([0, 1], [0, 1],linestyle='--')
        plt.title("ROC_AUC for CRT Score:"+str(crt_score))
        plt.legend(loc=4)
        plt.show()
        
show_roc_curve('all')    
#show_roc_curve(0)
#show_roc_curve(1)
#show_roc_curve(2)
#show_roc_curve(3)
#show_roc_curve(4)

## Figure

In [ ]:
# Data
r = [0,1,2,3,4]
raw_data = {'greenBars': tp, 'orangeBars': tn,'blueBars': fp,'redBars':fn}
df = pd.DataFrame(raw_data)
 
# From raw value to percentage
totals = [i+j+k+l for i,j,k,l in zip(df['greenBars'], df['orangeBars'], df['blueBars'],df['redBars'])]
greenBars = [i / j * 100 for i,j in zip(df['greenBars'], totals)]
orangeBars = [i / j * 100 for i,j in zip(df['orangeBars'], totals)]
blueBars = [i / j * 100 for i,j in zip(df['blueBars'], totals)]
redBars=[i / j * 100 for i,j in zip(df['redBars'], totals)]
# plot
barWidth = 0.85
names = ('0','1','2','3','4')
# Create green Bars
plt.bar(r, greenBars, color='#b5ffb9', edgecolor='white', width=barWidth, label="TP")
# Create orange Bars
plt.bar(r, orangeBars, bottom=greenBars, color='#f9bc86', edgecolor='white', width=barWidth, label="TN")
# Create blue Bars
plt.bar(r, blueBars, bottom=[i+j for i,j in zip(greenBars, orangeBars)], color='#a3acff', edgecolor='white', width=barWidth, label="FP")
plt.bar(r, redBars, bottom=[i+j+k for i,j,k in zip(greenBars, orangeBars, blueBars)], color='#ffcccb', edgecolor='white', width=barWidth, label="FN")
# Custom x axis
plt.xticks(r, names)
plt.xlabel("CRT Score")
plt.ylabel("% of Samples")
# Add a legend
plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
 
# Show graphic
plt.show()




## optimize features using RFE

In [ ]:
print(X_test)

In [ ]:
#export
filename = 'votingclf.model'
joblib.dump(clf, filename)

In [ ]:
type(X_test)

In [ ]:
import sklearn
import shap
from sklearn.model_selection import train_test_split


# print the JS visualization code to the notebook
shap.initjs()
explainer = shap.KernelExplainer(clf.predict_proba, X, link="logit")
shapvalues = explainer.shap_values(X)
#shap.summary_plot(explainer, X_test)
#shap_values = explainer.shap_values(X_test)

# plot the SHAP values for the Setosa output of the first instance
#shap.force_plot(explainer.expected_value[0], shap_values[0][0,:], X_test.iloc[0,:], link="logit")

In [ ]:
shap.summary_plot(shapvalues[0], X,plot_type='bar',max_display=X.shape[1])

In [ ]:
type(shapvalues)
#shap.plots.heatmap(shapvalues[1].toDF())
shap.force_plot(explainer.expected_value[1], shapvalues[1], X)

In [ ]:
print(X.columns)

In [ ]:
#feature_names = [i for i in X.columns if X[i].dtype in [np.int64, np.int64]]

#shapDF=pd.DataFrame(shapvalues[1],columns=X.columns)
#shap.plots.heatmap(shapvalues[0][:1000])
shap.plots.scatter(shapvalues[0])

In [ ]:
shap.summary_plot(shapvalues[1], X,show=False,max_display=X.shape[1])


In [ ]:
f.show()

In [ ]:
shap.dependence_plot('weight2',shapvalues[1], X,interaction_index='height2')

In [ ]:
import pickle
pickle.dump(shapvalues,open("shap-scrt3.p","wb"))
pickle.dump(explainer,open("exp-scrt3.p","wb"))